In [217]:
from pathlib import Path
from pysota.core import Publication
from pysota.process import Persistence

db: list[Publication] = Persistence.load_files(path=Path('../results/clustered/euclidean'), query_name='cluster_4')
print(len(db))

17


In [218]:
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer

# # Sample documents
# documents = [doc.abstract for doc in db]

# # Convert documents into a document-term matrix
# vectorizer = CountVectorizer(stop_words='english')
# dtm = vectorizer.fit_transform(documents)

# # Set the number of topics
# n_topics = 10

# # Initialize and fit the LDA model
# lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
# lda.fit(dtm)

# # Display the top words for each topic
# n_top_words = 3
# feature_names = vectorizer.get_feature_names_out()

# for topic_idx, topic in enumerate(lda.components_):
#     print(f"Topic {topic_idx}:")
#     top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
#     print(" ".join(top_features))


In [ ]:
from gensim import corpora
from gensim.models import LdaModel

import spacy


nlp = spacy.load('en_core_web_lg')

def preprocess(text):
    # exclude = ['llm', 'llms', 'trust', 'trustworthy', 'trustworthiness', 'ai', 'intelligence', 'data', 'datum']
    # exclude = ['relativity', 'quantum', 'theory', 'physic', 'physics']
    # exclude = ['reinforcement', 'learning', 'learn', 'algorithm']
    exclude = []
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha and token.lemma_ not in exclude
    ]
    return tokens


# Preprocess each document
texts = [preprocess(doc.abstract) for doc in db]


In [240]:

# Create a dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Filter out words that occur less than 2 documents, or more than 50% of the documents
# dictionary.filter_extremes(no_below=2)

# Train the LDA model
# Set training parameters
num_topics = 6
passes = 100

# Train the LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)



In [241]:

# Display topics
topics = lda_model.print_topics(num_words=5)
# topics = lda_model.show_topics()
for topic in topics:
    print(topic)



(0, '0.031*"ethic" + 0.028*"machine" + 0.020*"sl" + 0.016*"language" + 0.012*"review"')
(1, '0.022*"approach" + 0.020*"action" + 0.019*"state" + 0.015*"study" + 0.015*"space"')
(2, '0.013*"different" + 0.013*"challenge" + 0.013*"paper" + 0.013*"provide" + 0.013*"new"')
(3, '0.032*"model" + 0.024*"system" + 0.016*"state" + 0.013*"chapter" + 0.013*"work"')
(4, '0.002*"reward" + 0.002*"compare" + 0.002*"present" + 0.002*"action" + 0.002*"decision"')
(5, '0.023*"competition" + 0.015*"ai" + 0.015*"develop" + 0.012*"framework" + 0.012*"evaluation"')


In [242]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print(f'Coherence Score: {coherence_score}')


Coherence Score: 0.3794625177755074


In [243]:
import pandas as pd
from gensim.models import LdaModel

def lda_topics_to_dataframe(lda_model, num_words=10):
    # Extract topics from the LDA model
    topics = lda_model.show_topics(num_topics=-1, num_words=num_words, formatted=False)
    
    # Initialize a list to hold the parsed data
    data = []
    
    # Iterate over each topic
    for topic_num, terms in topics:
        for term, weight in terms:
            data.append([topic_num, term, weight])
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=['Topic', 'Term', 'Weight'])
    
    # Sort the DataFrame by Topic and Weight in descending order
    df = df.sort_values(by=['Topic', 'Weight'], ascending=[True, False]).reset_index(drop=True)
    
    return df

# Example usage
# Assuming you have an LdaModel object named 'lda_model'
df_topics = lda_topics_to_dataframe(lda_model, num_words=5)

In [244]:
for topic in df_topics.Topic.unique():
    terms = df_topics[df_topics.Topic == topic].Term.tolist()
    print(f"Topic {topic}: {', '.join(terms)} ")

Topic 0: ethic, machine, sl, language, review 
Topic 1: approach, action, state, study, space 
Topic 2: different, challenge, paper, provide, new 
Topic 3: model, system, state, chapter, work 
Topic 4: reward, compare, present, action, decision 
Topic 5: competition, ai, develop, framework, evaluation 
